<a href="https://colab.research.google.com/github/bobuallah/CG/blob/main/CG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🎮 THE ZERO-TOUCH CONSOLE (Install -> Play -> AutoSave)
# Runs everything. Backs up automatically every 10 minutes.

import os, subprocess, time, datetime

# --- CONFIGURATION ---
# Auto-save interval in seconds (600 = 10 minutes)
SAVE_INTERVAL = 600
DRIVE_ROOT = "/content/drive/MyDrive/CloudConsole_Data"
CONFIG_BACKUP = f"{DRIVE_ROOT}/Configs"
SAVE_BACKUP = f"{DRIVE_ROOT}/Saves"

# ==========================================
# 1. SETUP & RESTORE
# ==========================================
from google.colab import drive
if not os.path.exists('/content/drive'):
    print("🔌 Connecting to Drive...")
    drive.mount('/content/drive')

os.makedirs(CONFIG_BACKUP, exist_ok=True)
os.makedirs(SAVE_BACKUP, exist_ok=True)

print("🛠️ Installing System (This takes ~4 mins)...")
# Quietly install everything in the background
subprocess.run("dpkg --add-architecture i386 && apt-get update", shell=True)
pkgs = "steam xfce4 xfce4-terminal python3-pip xdotool libfuse2 aria2 dbus-x11"
subprocess.run(f"apt-get install -y {pkgs} > /dev/null 2>&1", shell=True)

# Install Heroic
if not os.path.exists("/usr/bin/heroic"):
    heroic_url = "https://github.com/Heroic-Games-Launcher/HeroicGamesLauncher/releases/download/v2.10.0/heroic_2.10.0_amd64.deb"
    subprocess.run(f"wget -q -O heroic.deb {heroic_url} && apt-get install -y ./heroic.deb > /dev/null 2>&1", shell=True)

# Install Ludusavi (The Auto-Saver)
if not os.path.exists("/usr/bin/ludusavi"):
    ludu_url = "https://github.com/mtkennerly/ludusavi/releases/download/v0.21.0/ludusavi-v0.21.0-linux.tar.gz"
    subprocess.run(f"wget -q -O ludusavi.tar.gz {ludu_url} && tar -xzf ludusavi.tar.gz && mv ludusavi /usr/bin/ && chmod +x /usr/bin/ludusavi", shell=True)

print("🔄 Restoring Logins & Configs...")
# Restore Steam
if os.path.exists(f"{CONFIG_BACKUP}/Steam_Config.tar.gz"):
    os.makedirs("/root/.local/share/Steam", exist_ok=True)
    subprocess.run(f"tar -xzf '{CONFIG_BACKUP}/Steam_Config.tar.gz' -C /", shell=True)

# Restore Heroic
if os.path.exists(f"{CONFIG_BACKUP}/Heroic_Config.tar.gz"):
    os.makedirs("/root/.config/heroic", exist_ok=True)
    subprocess.run(f"tar -xzf '{CONFIG_BACKUP}/Heroic_Config.tar.gz' -C /", shell=True)

# Restore Saves
if os.path.exists(f"{SAVE_BACKUP}/Latest_Saves.zip"):
    subprocess.run(f"ludusavi restore --force --path '{SAVE_BACKUP}/Latest_Saves.zip'", shell=True)

# ==========================================
# 2. LAUNCH SERVICES
# ==========================================
print("🚀 Launching VNC & Steam...")
subprocess.Popen(["vncserver", ":1", "-geometry", "1280x720", "-depth", "24"])
os.environ["DISPLAY"] = ":1"
subprocess.Popen(["steam", "-bigpicture", "-no-browser", "-silent"])

# Cloudflare Tunnel
if not os.path.exists("/usr/bin/cloudflared"):
    subprocess.run("wget -q -O cloudflared-linux-amd64 https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", shell=True)
    subprocess.run("chmod +x cloudflared-linux-amd64 && mv cloudflared-linux-amd64 /usr/bin/cloudflared", shell=True)

# Start Tunnel and grab URL
print("🔗 GENERATING LINK...")
tunnel_process = subprocess.Popen(["cloudflared", "tunnel", "--url", "vnc://localhost:5901"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# ==========================================
# 3. THE INFINITE AUTO-SAVE LOOP
# ==========================================
print(f"✅ SYSTEM LIVE! Auto-saving every {SAVE_INTERVAL/60} minutes.")
print("⚠️ Check the logs below for the Connection Link.")

last_backup = time.time()

while True:
    # 1. Print Tunnel Output (So you can see the link)
    line = tunnel_process.stderr.readline()
    if line:
        decoded_line = line.decode('utf-8').strip()
        if "trycloudflare.com" in decoded_line:
            print(f"🔗 CLICK HERE TO PLAY: {decoded_line}")

    # 2. Check time for Backup
    if time.time() - last_backup > SAVE_INTERVAL:
        print(f"💾 [Auto-Save] Backing up data at {datetime.datetime.now().strftime('%H:%M')}...")

        # Backup Saves
        subprocess.run(f"ludusavi backup --force --path '{SAVE_BACKUP}/Latest_Saves.zip'", shell=True)

        # Backup Steam Login (In case it changed)
        steam_paths = ["/root/.local/share/Steam/config", "/root/.local/share/Steam/userdata"]
        tar_cmd = f"tar -czf '{CONFIG_BACKUP}/Steam_Config.tar.gz' " + " ".join(steam_paths)
        subprocess.run(tar_cmd, shell=True)

        # Backup Heroic Login
        if os.path.exists("/root/.config/heroic"):
            subprocess.run(f"tar -czf '{CONFIG_BACKUP}/Heroic_Config.tar.gz' '/root/.config/heroic'", shell=True)

        print("✅ [Auto-Save] Complete. Your progress is safe.")
        last_backup = time.time()

    # Small sleep to prevent CPU spiking
    time.sleep(1)